# Coherence Analyses

This notebook contains example of how we can use coherence analyses to identify coherent signals in seismic data. We use data from Brady's Hot Spring geothermal field in Nevada, USA. The data contains both coherent signals (microseismic events) and incoherent noise (background seismic noise). We will compare the performance of different methods for identifying coherent signals, including exact eigenvalue decomposition, SVD approximation, and QR approximation.

## Data

The data used in this notebook can be downloaded via the [AWS S3 Explorer for the Open Energy Data Initiative](https://data.openei.org/s3_viewer?bucket=nrel-pds-porotomo&prefix=DAS%2FH5%2FDASH%2F). The data is stored in the `nrel-pds-porotomo` bucket and the `DAS/H5/DASH/` prefix. There are separated by dates of recording and arranged in chronological order. The data is stored in the Hierarchical Data Format (HDF) and can be read using the `h5py` package. The file names end in the format `YYMMDDHHmmss.h5` where `YY` is the year, `MM` is the month, `DD` is the day, `HH` is the hour, `mm` is the minute, and `ss` is the second. 

### Make imports and write functions

In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.join(os.path.dirname(""), os.pardir, os.pardir))
import coherence_analysis.utils as f

### Parameters for data and tests

In [2]:
samples_per_sec = 1000
fsize = 15
tick_size = 12
nsensors = 200

start_ch = 3100
nchannels = 2000

win_len = 2.5
overlap = 0

colors = ["#800000", "#FFD700", "#663399", "#000000", "#008B8B"]

# Background noise test

Here, we load this data with a custom function. The command line interface available with `coherence_analyses.coherence_analyses` loads the data with `dascore` package, and hence is able to read a range of DAS data formats.

In [3]:
file = r"D:\CSM\Mines_Research\Test_data\Brady_Hotspring\PoroTomo_iDAS16043_160312000018.h5"
data, _ = f.load_brady_hdf5(file, normalize="no")

file = r"D:\CSM\Mines_Research\Test_data\Brady_Hotspring\PoroTomo_iDAS16043_160312000048.h5"
data2, _ = f.load_brady_hdf5(file, normalize="no")

data_noise = np.append(data, data2, axis=1)

file = r"D:\CSM\Mines_Research\Test_data\Brady_Hotspring\PoroTomo_iDAS16043_160312000118.h5"
data2, _ = f.load_brady_hdf5(file, normalize="no")

data_noise = np.append(data_noise, data2[:, :10000], axis=1)
data_noise = data_noise[
    start_ch : nchannels + start_ch : int(nchannels / nsensors)
]

Calculate the detection parameter,

$$\frac{\lambda_i}{\sum_{i=1}^{n}{\lambda_i}}$$

at each frequency. Here $\lambda_i$ is the $i^{th}$ eigenvalue of the coherence matrix.

In [4]:
detection_sig_exact, eigs_exact, freqs = f.coherence(
    data_noise,
    win_len,
    overlap,
    resolution=1,
    sample_interval=1 / samples_per_sec,
    method="exact",
    max_freq=300,
)
detection_sig_svd, eigs_svd, freqs = f.coherence(
    data_noise,
    win_len,
    overlap,
    resolution=1,
    sample_interval=1 / samples_per_sec,
    method="svd",
    max_freq=300,
)
detection_sig_qr, eigs_qr, freqs = f.coherence(
    data_noise,
    win_len,
    overlap,
    resolution=1,
    sample_interval=1 / samples_per_sec,
    method="qr",
    max_freq=300,
)

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(
    freqs[1:],
    detection_sig_qr[1:],
    "--*",
    markersize=5,
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    freqs[1:],
    detection_sig_svd[1:],
    "--s",
    markersize=5,
    label="SVD approximation",
)
plt.plot(
    freqs[1:],
    detection_sig_exact[1:],
    "--o",
    markersize=5,
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel(r"$\frac{\lambda_1}{\sum_{i=1}^{n}{\lambda_i}}$", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
# plt.title("Proportion of $\lambda_1$ in sum of all eigenvalues",fontsize=fsize)
plt.title("Background Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

### Plot data

In [ ]:
v_min = -0.015
v_max = 0.015

fig2 = plt.figure()
img2 = plt.imshow(
    data_noise[start_ch : nchannels + start_ch],
    cmap="RdBu",
    vmin=v_min,
    vmax=v_max,
    aspect="auto",
    interpolation="none",
    extent=(0, 70, nchannels + start_ch, start_ch),
)

plt.xlabel("Time (seconds)", fontsize=fsize)
plt.ylabel("Channels", fontsize=fsize)
# plt.title('Background noise',fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=tick_size)

# Microseismic event

In [ ]:
file = r"D:\CSM\Mines_Research\Test_data\Brady_Hotspring\PoroTomo_iDAS16043_160314083818.h5"
data, _ = f.load_brady_hdf5(file, normalize="no")

file = r"D:\CSM\Mines_Research\Test_data\Brady_Hotspring\PoroTomo_iDAS16043_160314083848.h5"
data2, _ = f.load_brady_hdf5(file, normalize="no")

data = np.append(data, data2, axis=1)

file = r"D:\CSM\Mines_Research\Test_data\Brady_Hotspring\PoroTomo_iDAS16043_160314083918.h5"
data2, _ = f.load_brady_hdf5(file, normalize="no")

data = np.append(data, data2[:, :10000], axis=1)
data = data[start_ch : nchannels + start_ch : int(nchannels / nsensors)]

In [ ]:
fig2 = plt.figure()
img2 = plt.imshow(
    data[start_ch : nchannels + start_ch],
    cmap="RdBu",
    vmin=v_min,
    vmax=v_max,
    aspect="auto",
    interpolation="none",
    extent=(0, 70, nchannels + start_ch, start_ch),
)

plt.xlabel("Time (seconds)", fontsize=fsize)
plt.ylabel("Sensors", fontsize=fsize)

plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=tick_size)

In [ ]:
detection_sig_exact2, eigs_exact2, freqs = f.coherence(
    data,
    win_len,
    overlap,
    resolution=1,
    sample_interval=1 / samples_per_sec,
    method="exact",
    max_freq=300,
)
detection_sig_svd2, eigs_svd2, freqs = f.coherence(
    data,
    win_len,
    overlap,
    resolution=1,
    sample_interval=1 / samples_per_sec,
    method="svd",
    max_freq=300,
)
detection_sig_qr2, eigs_qr2, freqs = f.coherence(
    data,
    win_len,
    overlap,
    resolution=1,
    sample_interval=1 / samples_per_sec,
    method="qr",
    max_freq=300,
)

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(
    freqs[1:],
    detection_sig_qr2[1:],
    "--*",
    markersize=5,
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    freqs[1:],
    detection_sig_svd2[1:],
    "--s",
    markersize=5,
    label="SVD approximation",
)
plt.plot(
    freqs[1:],
    detection_sig_exact2[1:],
    "--o",
    markersize=5,
    color="darkviolet",
    alpha=0.6,
    label="Exact",
)

plt.ylabel(r"$\frac{\lambda_1}{\sum_{i=1}^{n}{\lambda_i}}$", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
# plt.title("Proportion of $\lambda_1$ in sum of all eigenvalues",fontsize=fsize)
plt.title("Microseismic Event", fontsize=fsize)
plt.legend(fontsize=fsize)

## Signal and noise summary comparison

The goal is to get a general overview within all the frequency range of interest. In the following plot, we look at how the various methods perform in this regard.

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
plt.plot(
    freqs[1:],
    detection_sig_exact2[1:],
    color="darkviolet",
    label="Microseismic Event",
)
plt.plot(
    freqs[1:],
    detection_sig_exact[1:],
    color="goldenrod",
    alpha=0.9,
    label="Background Noise",
)

# plt.ylabel(r"$\frac{\lambda_1}{\sum_{i=1}^{n}{\lambda_i}}$",fontsize=fsize)
plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.ylim(0, 1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.title("Exact", fontsize=fsize)
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
plt.plot(
    freqs[1:],
    detection_sig_svd2[1:],
    color="darkviolet",
    label="Microseismic Event",
)
plt.plot(
    freqs[1:],
    detection_sig_svd[1:],
    color="goldenrod",
    alpha=0.9,
    label="Background Noise",
)
plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.ylim(0, 1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.title("SVD", fontsize=fsize)
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
plt.plot(
    freqs[1:],
    detection_sig_qr2[1:],
    color="darkviolet",
    label="Microseismic Event",
)
plt.plot(
    freqs[1:],
    detection_sig_qr[1:],
    color="goldenrod",
    alpha=0.9,
    label="Background Noise",
)
plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.ylim(0, 1)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.title("QR", fontsize=fsize)
plt.legend(fontsize=tick_size)

## Decay of eigenvalues corresponding to particular frequency

Beyond the summary over the entire frequency range, we can also look at the decay of eigenvalues corresponding to a particular frequency. Here, we choose a frequency of 14 Hz.

In [ ]:
i = 35
plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
plt.plot(
    np.sort(eigs_exact2[i] / np.sum(eigs_exact2[i]))[::-1],
    "-o",
    color="darkviolet",
    label="Coherent Signal",
)
plt.plot(
    np.sort(eigs_exact[i] / np.sum(eigs_exact[i]))[::-1],
    "-*",
    color="goldenrod",
    label="Background Noise",
)

plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Index, i", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.ylim([-0.05, 0.9])
plt.legend(fontsize=tick_size)
plt.title("Exact", fontsize=fsize)

plt.subplot(1, 3, 2)
plt.plot(
    np.sort(eigs_svd2[i] / np.sum(eigs_svd2[i]))[::-1],
    "-o",
    color="darkviolet",
    label="Coherent Signal",
)
plt.plot(
    np.sort(eigs_svd[i] / np.sum(eigs_svd[i]))[::-1],
    "-*",
    color="goldenrod",
    label="Background Noise",
)

plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Index, i", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.ylim([-0.05, 0.9])
plt.legend(fontsize=tick_size)
plt.title("SVD approximation", fontsize=fsize)

plt.subplot(1, 3, 3)
plt.plot(
    np.sort(eigs_qr2[i] / np.sum(eigs_qr2[i]))[::-1],
    # np.sort(eigenvals2 / np.sum(eigenvals2))[::-1],
    "-o",
    color="darkviolet",
    label="Coherent Signal",
)
plt.plot(
    np.sort(eigs_qr[i] / np.sum(eigs_qr[i]))[::-1],
    # np.sort(eigenvals / np.sum(eigenvals))[::-1],
    "-*",
    color="goldenrod",
    label="Background Noise",
)

plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Index, i", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.ylim([-0.05, 0.9])
plt.legend(fontsize=tick_size)
plt.title("QR approximation", fontsize=fsize)